<a href="https://colab.research.google.com/github/arifinnasif/Natural-Hazard-Prediction/blob/master/lightnet_on_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch.nn as nn
import torch
import random
from tqdm import tqdm

## ConvLSTM

In [21]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis
        # print("size of h_cur", h_cur.size())
        # print("size of combined", combined.size())

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [22]:
conv_lstm = ConvLSTM(1,[8],[(5,5)],1, batch_first=True) # 1 channel, 8 hidden channel (from geng), 5x5 kernel, 1 layer(?), batch first -> batch er size input tensor e first e dicci,
x = torch.rand((32, 6, 1, 25, 25)) # 32 samples in a batch (batch first), prev 6 hours, 1 channel, 25x25 grid
# convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
_, last_states = conv_lstm(x)
h,c = last_states[0]  # 0 for layer index, 0 for h index
# print(last_states[0][1].size())
# print("-----")
# print(h.size())


In [23]:
x_ = torch.rand((32, 6, 50, 50))
conv_ = nn.Conv2d(in_channels = 6, out_channels = 6*4, kernel_size = 5, groups = 6, stride = 2, padding = 2)
# conv_.weight.data = [None, None, ...].repeat(64, 1, 1, 1)
# print(conv_.weight.data.size())
conv_(x_).size()

torch.Size([32, 24, 25, 25])

In [24]:
class Encoder_old(nn.Module):
  def __init__(self):
    super(Encoder_old, self).__init__()
    self.prev_hours = 6
    self.conv_2 = nn.Conv2d(in_channels = self.prev_hours,
                          out_channels = self.prev_hours*4, # each input frame gets mapped to 4 layer
                          groups = self.prev_hours,
                          kernel_size = 7,
                          stride = 2,
                          padding = 3)
    self.conv_lstm = ConvLSTM(input_dim = 4,
                               hidden_dim = [8],
                               kernel_size = [(5,5)],
                               num_layers = 1,
                               batch_first=True)
    # print(self.conv_2.weight.data.size())

  def forward(self, input_tensor):
    x = self.conv_2(input_tensor.flatten(1,2))
    _, last_states = self.conv_lstm(torch.unflatten(x, dim = 1, sizes = (6, 4)))
    h,c = last_states[0]

    return h,c





In [25]:
class Encoder(nn.Module):
  def __init__(self, in_channels_for_a_given_time, prev_hours):
    super(Encoder, self).__init__()
    self.prev_hours = prev_hours
    self.hidden_dim = 8
    self.stride = 2
    self.in_channels_for_a_given_time = in_channels_for_a_given_time
    self.conv_2 = nn.Conv2d(in_channels = self.in_channels_for_a_given_time, # one 2d lightnig grid at time t
                          out_channels = 4, # from geng
                          kernel_size = 7,
                          stride = self.stride,
                          padding = 3)
    self.conv_lstm_cell = ConvLSTMCell(input_dim = 4,
                               hidden_dim = self.hidden_dim,
                               kernel_size = (5,5),
                               bias=True)
    # print(self.conv_2.weight.data.size())

  def forward(self, input_tensor):
    # x = self.conv_2(input_tensor.flatten(1,2))
    # _, last_states = self.conv_lstm(torch.unflatten(x, dim = 1, sizes = (6, 4)))
    # h,c = last_states[0]
    b, prev_hours, channels, height, width = input_tensor.size()
    h,c = self.init_hidden(batch_size=b, image_size=(height // self.stride, width // self.stride))

    for t in range(prev_hours):
      x = self.conv_2(input_tensor[:, t, :, :, :])
      h, c = self.conv_lstm_cell(x, cur_state=[h, c])
      return h,c

    #####

    #     layer_output_list = []
    #     last_state_list = []

    #     seq_len = input_tensor.size(1)
    #     cur_layer_input = input_tensor

    #     for layer_idx in range(self.num_layers):

    #         h, c = hidden_state[layer_idx]
    #         output_inner = []
    #         for t in range(seq_len):
    #             h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
    #                                              cur_state=[h, c])
    #             output_inner.append(h)

    #         layer_output = torch.stack(output_inner, dim=1)
    #         cur_layer_input = layer_output

    #         layer_output_list.append(layer_output)
    #         last_state_list.append([h, c])

    #     if not self.return_all_layers:
    #         layer_output_list = layer_output_list[-1:]
    #         last_state_list = last_state_list[-1:]

    #     return layer_output_list, last_state_list

    # return h,c

  def init_hidden(self, batch_size, image_size):
      height, width = image_size
      return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv_2.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv_2.weight.device))





In [26]:
class Fusion(nn.Module):
  def __init__(self, total_in_channels):
    super(Fusion,self).__init__()
    # h_stacked = torch.cat(h_list, dim = 1)
    # sc_stacked = torch.cat(c_list, dim = 1)
    self.conv_3 = nn.Conv2d(in_channels = total_in_channels,
                            out_channels = 64, # from the paper
                            kernel_size=1)
    self.conv_4 = nn.Conv2d(in_channels = total_in_channels,
                            out_channels = 64, # from the paper
                            kernel_size=1)

  def forward(self, h_list, c_list):
    h_stacked = torch.cat(h_list, dim = 1)
    c_stacked = torch.cat(c_list, dim = 1)


    h_fused_conved = self.conv_3(h_stacked)
    c_fused_conved = self.conv_4(c_stacked)

    return h_fused_conved, c_fused_conved


In [27]:
class Decoder(nn.Module):
  def __init__(self, in_channels_for_a_given_time, next_hours):
    super(Decoder, self).__init__()
    self.next_hours = next_hours
    self.in_channels_for_a_given_time = in_channels_for_a_given_time
    self.conv_5 = nn.Conv2d(in_channels = self.in_channels_for_a_given_time, # single L_{-1} frame
                          out_channels = 4, # from geng
                          kernel_size = 7,
                          stride = 2,
                          padding = 3)
    self.conv_lstm_cell = ConvLSTMCell(input_dim=4, # output of conv
                                       hidden_dim=64, # from geng
                                       kernel_size=(5,5), # from geng
                                       bias=True)
    # print(self.conv_2.weight.data.size())
    self.deconv = nn.ConvTranspose2d(in_channels=64, # as previous conv_lstm_cell had 64 hidden dim so the size is [:,64,:,:]
                                     out_channels=64, # from geng
                                     kernel_size = 7, # from geng
                                     stride = 2, # from geng
                                     padding = 3,
                                     output_padding = 1) # not sure

    self.conv_6 = nn.Conv2d(in_channels = 64, # previous deconv
                          out_channels = 1, # as it is just one layer
                          kernel_size = 1, # from geng
                          stride = 1) # from geng

  def forward(self, input_tensor_L_neg_1, h, c):
    # x = self.conv_2(input_tensor.flatten(1,2))
    # _, last_states = self.conv_lstm(torch.unflatten(x, dim = 1, sizes = (6, 4)))
    # h,c = last_states[0]
    b, _, height, width = input_tensor_L_neg_1.size()

    pred_output = []

    for i in range(self.next_hours):
      # print("---")
      x = self.conv_5(input_tensor_L_neg_1)
      # print(x.size())
      # print("dbg", x.size())
      h, c = self.conv_lstm_cell(x, cur_state=[h, c])
      # print(h.size())
      x = self.deconv(h)
      # print(x.size())
      x = self.conv_6(x)
      # print(x.size())

      pred_output.append(x)
      # print("---")

    pred_output = torch.stack(pred_output, 0).permute(1, 0, 2, 3, 4)
    # print(torch.stack(pred_output, 0).size())





    return pred_output



In [28]:
class LightNet(nn.Module):
  def __init__(self):
    super(LightNet, self).__init__()
    self.obs_enc = Encoder(in_channels_for_a_given_time=1,
                           prev_hours=6)
    self.fus = Fusion(8) # total h (or c) channels. as we are going to use just one h from the obs encoder (size of h = [:,8,:,:])
    self.pred_dec = Decoder(in_channels_for_a_given_time=1,
                            next_hours=6)

  def forward(self, input_tensor):
    h,c = self.obs_enc(input_tensor)
    h,c = self.fus([h],[c])
    out = self.pred_dec(input_tensor[:,-1,:,:,:], h, c)

    return out

In [29]:
# model = LightNet()
# x = torch.rand((32, 6, 1, 50, 50)) # [batch_size, prev_hours, input_layer, image_height, image_width]
# model(x)

In [30]:
def MeteorologicalMeasures(output, target):
  output, target = output.to(torch.device("cuda")), target.to(torch.device("cuda"))
  true_pos = 0
  true_neg = 0

  false_pos = 0
  false_neg = 0

  ytrue = target
  ypred = torch.sigmoid(output)
  ypred = torch.round(ypred)
  true_positives = torch.sum(ytrue * ypred)
  possible_positives = torch.sum(ytrue)
  POD = true_positives / (possible_positives + 1e-10)
  predicted_positives = torch.sum(ypred)
  FAR = true_positives / (predicted_positives + 1e-10)
  return POD, FAR



In [34]:
def train(model):
  data = torch.rand((1024,1,50,50), dtype=torch.double)
  data[data>0.5] = 1.0
  data[data<0.5] = 0.0
  # print(data)
  train_pct = 0.7
  prev_hours = 6
  next_hours = 6
  batch_size = 32
  start_index_list = list(range(data.size(0) - prev_hours - next_hours))
  random.shuffle(start_index_list)
  train_start_index_list = start_index_list[:int(len(start_index_list)*train_pct)]
  val_start_index_list = start_index_list[int(len(start_index_list)*train_pct):]
  # print("hello")

  # print(len(train_start_index_list))
  # print(len(val_start_index_list))

  ## train set creation
  train_x_set = []
  train_y_set = []
  for i in range(len(train_start_index_list) // batch_size):
    train_x_batch = []
    train_y_batch = []
    for j in range(batch_size):
      start = train_start_index_list[batch_size * i + j]
      train_x_batch.append(data[start:(start+prev_hours),:,:,:])
      train_y_batch.append(data[(start+prev_hours):(start+prev_hours+next_hours),:,:,:])

    train_x_batch = torch.stack(train_x_batch,0)
    train_y_batch = torch.stack(train_y_batch,0)

    train_x_set.append(train_x_batch)
    train_y_set.append(train_y_batch)

  # train_x_set = torch.stack(train_x_set,0)
  # train_y_set = torch.stack(train_y_set,0)
  # print(train_x_set.size())
  # print(train_y_set.size())

  ## validation set creation
  val_x_set = []
  val_y_set = []
  for i in range(len(val_start_index_list) // batch_size):
    val_x_batch = []
    val_y_batch = []
    for j in range(batch_size):
      start = val_start_index_list[batch_size * i + j]
      val_x_batch.append(data[start:(start+prev_hours),:,:,:])
      val_y_batch.append(data[(start+prev_hours):(start+prev_hours+next_hours),:,:,:])

    val_x_batch = torch.stack(val_x_batch,0)
    val_y_batch = torch.stack(val_y_batch,0)

    val_x_set.append(val_x_batch)
    val_y_set.append(val_y_batch)

  # val_x_set = torch.stack(val_x_set,0)
  # val_y_set = torch.stack(val_y_set,0)
  # print(val_x_set.size())
  # print(val_y_set.size())
  # print(data[start:(start+prev_hours+next_hours),:,:,:].size())

  model.double()
  # print(model(train_x_set[0]).size())

  ce_loss = nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  epoch = 50

  for e in range(epoch):
    avg_train_loss = 0
    avg_val_loss = 0
    avg_train_POD = 0
    avg_train_FAR = 0
    avg_train_ETS = 0
    avg_val_POD = 0
    avg_val_FAR = 0
    avg_val_ETS = 0
    print("Epoch [",e+1,"/",epoch,"]", end=" ")
    for i in tqdm(range(len(train_x_set))):
      data, target = train_x_set[i].to(torch.device("cuda")), train_y_set[i].to(torch.device("cuda"))
      optimizer.zero_grad()
      # train
      model.train()
      output = model(data)
      train_loss = ce_loss(output, target)
      train_POD, train_FAR = MeteorologicalMeasures(output, target)
      avg_train_loss = avg_train_loss + train_loss.item()
      avg_train_POD = avg_train_POD + train_POD.item()
      avg_train_FAR = avg_train_FAR + train_FAR.item()
      # avg_train_ETS = avg_train_ETS + train_ETS
      train_loss.backward()
      optimizer.step()

    for i in range(len(val_x_set)):
      data, target = val_x_set[i].to(torch.device("cuda")), val_y_set[i].to(torch.device("cuda"))
      # val
      model.eval()
      output = model(data)
      val_loss = ce_loss(output, target)
      val_POD, val_FAR = MeteorologicalMeasures(output, target)
      avg_val_loss = avg_val_loss + val_loss.item()
      avg_val_POD = avg_val_POD + val_POD.item()
      avg_val_FAR = avg_val_FAR + val_FAR.item()
      # avg_val_ETS = avg_val_ETS + val_ETS
      # train_loss.backward()
      # optimizer.step()
    avg_train_loss = avg_train_loss / len(train_x_set)
    avg_train_POD = avg_train_POD / len(train_x_set)
    avg_train_FAR = avg_train_FAR / len(train_x_set)
    avg_train_ETS = avg_train_ETS / len(train_x_set)

    avg_val_loss = avg_val_loss / len(val_x_set)
    avg_val_POD = avg_val_POD / len(val_x_set)
    avg_val_FAR = avg_val_FAR / len(val_x_set)
    avg_val_ETS = avg_val_ETS / len(val_x_set)
    print("train loss", avg_train_loss,"|", "train POD", avg_train_POD,"|", "train FAR", avg_train_FAR)#,"|", "train ETS", avg_train_ETS)
    print("val loss", avg_val_loss,"|", "val POD", avg_val_POD,"|", "val FAR", avg_val_FAR)#,"|", "val ETS", avg_val_ETS)
    print()
      # if batch_idx % args.log_interval == 0:
      # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
      #             epoch, i * len(data), len(train_loader.dataset),
      #             100. * batch_idx / len(train_loader), loss.item()))
      #         if args.dry_run:
      #             break
      # for i in range(opt.steps):
      #     print('STEP: ', i)
      #     def closure():
      #         optimizer.zero_grad()
      #         out = seq(input)
      #         loss = criterion(out, target)
      #         print('loss:', loss.item())
      #         loss.backward()
      #         return loss
      #     optimizer.step(closure)


model = LightNet()
model.to(torch.device('cuda'))
train(model)


Epoch [ 1 / 50 ] 

100%|██████████| 22/22 [01:11<00:00,  3.27s/it]


train loss 5.374536001660926 | train POD 0.0 | train FAR 0.0
val loss 5.371370665762395 | val POD 0.0 | val FAR 0.0

Epoch [ 2 / 50 ] 

100%|██████████| 22/22 [01:11<00:00,  3.26s/it]


train loss 5.37452006041462 | train POD 0.0 | train FAR 0.0
val loss 5.3713737655592455 | val POD 0.0 | val FAR 0.0

Epoch [ 3 / 50 ] 

...

train loss 5.369244667131117 | train POD 0.1740101022423082 | train FAR 0.5187252235623757
val loss 5.376434206493818 | val POD 0.1735603973231121 | val FAR 0.49952075618518854

Epoch [ 49 / 50 ] 

100%|██████████| 22/22 [01:11<00:00,  3.26s/it]


train loss 5.368998569992329 | train POD 0.17878643620238702 | train FAR 0.5189427076869021
val loss 5.3766798279461305 | val POD 0.18352876931009499 | val FAR 0.4994182998088713

Epoch [ 50 / 50 ] 

100%|██████████| 22/22 [01:11<00:00,  3.26s/it]


train loss 5.368753049891968 | train POD 0.1832036260217417 | train FAR 0.5189567236874374
val loss 5.37694528491005 | val POD 0.19254363573932448 | val FAR 0.499462092090679



In [ ]:
# x = torch.rand((32, 6, 1, 50, 50))
# enc = Encoder()
# h,c = enc(x)
# # print(h.size(1)+c.size(1))
# fus = Fusion(h.size(1))
# h,c = fus([h], [c])
# dec = Decoder()
# print("size of x[:,-1,:,:,:]", x[:,-1,:,:,:].size())
# print(h.size())
# out = dec(x[:,-1,:,:,:], h,c)
# print(h.size())
# print(c.size())
# print(torch.cat([h,c], dim=1).size())